In [15]:
import metaspore as ms

app_name='Swing Demo'
local=False
worker_count = 4
server_count = 4
batch_size=512
worker_memory='5G'
server_memory='5G'
coordinator_memory='5G'

spark_confs={
    "spark.network.timeout":"500",
    "spark.submit.pyFiles":"python.zip",
}

spark = ms.spark.get_session(local=local,
                             app_name=app_name,
                             batch_size=batch_size,
                             worker_count=worker_count,
                             server_count=server_count,
                             worker_memory=worker_memory,
                             server_memory=server_memory,
                             coordinator_memory=coordinator_memory,
                             spark_confs=spark_confs)
sc = spark.sparkContext
print(sc.version)
print(sc.applicationId)
print(sc.uiWebUrl)

3.1.2
spark-application-1644402952974
http://jupyter.my.nginx.test/hub/user-redirect/proxy/4040/jobs/


In [17]:
train_path='s3://dmetasoul-bucket/demo/movielens/match/num_negs_100/train.parquet'
test_path='s3://dmetasoul-bucket/demo/movielens/match/num_negs_100/test.parquet'
item_path='s3://dmetasoul-bucket/demo/movielens/match/num_negs_100/item.parquet'

train_dataset = spark.read.parquet(train_path)
train_dataset=train_dataset[train_dataset['label']==1]
test_dataset = spark.read.parquet(test_path)

In [18]:
train_dataset.show(10)

+-----+-------+------+---+----------+----------+--------+--------------------+--------------------+----------+--------------------+
|label|user_id|gender|age|occupation|       zip|movie_id|    recent_movie_ids|               genre|last_movie|          last_genre|
+-----+-------+------+---+----------+----------+--------+--------------------+--------------------+----------+--------------------+
|    1|   5638|     M| 18|         1|     32601|     266|2164712711268...|DramaRomanceWar...|      3591|        ComedyDrama|
|    1|   1440|     M| 35|        12|     94520|    2529|28191955195334...|       ActionSci-Fi|      2877|       DramaMusical|
|    1|   1010|     M| 25|         0|     10310|    2371|11862150592240...|              Comedy|      2352|        ComedyDrama|
|    1|      6|     F| 50|         9|     55117|     266|19479143604102...|DramaRomanceWar...|       368|ActionComedyWes...|
|    1|   4021|     M| 50|        20|     95236|    2193|35192826205810..

In [19]:
## Using Swing implementation in MindAlpha
estimator=ms.SwingEstimator(user_id_column_name='user_id',
                            item_id_column_name='movie_id',
                            behavior_column_name='label',
                            behavior_filter_value='1',
                            cassandra_catalog='',
                            cassandra_host_ip='',
                            cassandra_port=None,
                            cassandra_db_name='',
                            cassandra_table_name='',
)

model = estimator.fit(train_dataset)

In [20]:
model.df

DataFrame[key: string, value: array<struct<_1:string,_2:float>>]

In [30]:
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import RankingMetrics

user_col_name='user_id'
item_col_name='movie_id'
last_item_col_name='last_movie'

#  +---------+--------------------+
#  |  item_id|         label_items|
#  +---------+--------------------+
#  |209102738|[267044868, 31773...|
#  |198213284|[341389015, 25980...|
test_df = test_dataset.select(user_col_name, last_item_col_name, item_col_name)\
            .groupBy(user_col_name, last_item_col_name)\
            .agg(F.collect_set(item_col_name).alias('label_items'))

In [31]:
test_df = test_df.withColumnRenamed('last_movie','movie_id')

In [32]:
label_df = test_df
#  +---------+--------------------+---------+--------------------+
#  |  item_id|         label_items|      key|               value|
#  +---------+--------------------+---------+--------------------+
#  |163372107|[56871393, 177544...|     null|                null|
#  |176556528|[403809425, 43982...|176556528|[{179642423, 0.00...|
prediction_label_df = model.transform(label_df)

In [34]:
prediction_label_df.show(10)

+-------+--------+-----------+----+--------------------+
|user_id|movie_id|label_items| key|               value|
+-------+--------+-----------+----+--------------------+
|   3084|    1090|     [2529]|1090|[{1196, 61.55922}...|
|   4525|    1090|     [2890]|1090|[{1196, 61.55922}...|
|   3784|    1090|     [1358]|1090|[{1196, 61.55922}...|
|     57|    1090|     [1242]|1090|[{1196, 61.55922}...|
|    769|    1090|     [1304]|1090|[{1196, 61.55922}...|
|   2653|    1090|      [524]|1090|[{1196, 61.55922}...|
|   4693|    1090|     [1263]|1090|[{1196, 61.55922}...|
|    770|    1090|     [3359]|1090|[{1196, 61.55922}...|
|    601|    1090|     [2106]|1090|[{1196, 61.55922}...|
|   1352|    1572|     [2131]|1572|[{858, 0.00908822...|
+-------+--------+-----------+----+--------------------+
only showing top 10 rows



In [35]:
prediction_label_rdd = prediction_label_df.rdd.map(lambda x:(\
                                    [xx._1 for xx in x.value] if x.value is not None else [], \
                                     x.label_items))

In [36]:
metrics = RankingMetrics(prediction_label_rdd)

22/02/09 11:32:44 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.KubernetesClientException: too old resource version: 21004921 (21015749)
	at io.fabric8.kubernetes.client.dsl.internal.WatchConnectionManager$1.onMessage(WatchConnectionManager.java:258)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(RealCall.java:203)
	at okhttp3.internal.NamedRunnable.run(NamedRunnable.java:32)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent

In [37]:
print("Debug -- Precision@20: ", metrics.precisionAt(20))
print("Debug -- Recall@20: ", metrics.recallAt(20))
print("Debug -- MAP@20: ", metrics.meanAveragePrecisionAt(20))

Debug -- Precision@20:  0.005761589403973511


Debug -- Recall@20:  0.11523178807947022


Debug -- MAP@20:  0.024224720990804423
